# Implement Epsilon Greedy algorithm (DQN PyTorch Beginners Tutorial 4, by Johny Code)

Link to the Youtube video tutorial: https://www.youtube.com/watch?v=vYRpJo-KMSw&list=PL58zEckBH8fCMIVzQCRSZVPUp3ZAVagWi&index=5 <br />
Link to the Gymnasium official website: https://gymnasium.farama.org/ <br />
Link to the Flappy Bird environment GitHub repository: https://github.com/markub3327/flappy-bird-gymnasium <br />


**Information of Epsilon Greedy algorithm:**
1) Epsilon Greedy algorithm is a simple method to balance exploration and exploitation by choosing between exploration and exploitation randomly.
The epsilon-greedy, where epsilon refers to the probability of choosing to explore, exploits most of the time with a small chance of exploring.
    1) <img src="hidden\photo1.png" alt="This image is a representation of the simple neural network" style="width: 600px;"/>  <br />

Hyperparameters for Epsilon Greedy algorithm in hyperparameters.yml

In [ ]:
  epsilon_init: 1 # Starting here are the hyperparamters for Epsilon Greedy algorithm: we start at epsilon_init = 1, means there is a 100% chance the agent will take a random action and 0% chance the agent will take the action dictated by the trained policy or the policy that is being trained (represented by the DQN).
  epsilon_decay: 0.9995 # then slowly decrease epsilon 
  epsilon_min: 0.05 # all the way down to 0.05, means there is a 5% chance the agent will take a random action and 95% chance the agent will take the action dictated by the trained policy or the policy that is being trained (represented by the DQN).


The codes related to implementing Epsilon Greedy algorithm in the main file (agent.py)

In [ ]:
while not terminated: # This is an infinite loop for the agent(flappy bird) to interact with the environment through action, observation, and reward.
                # Next action:
                # (feed the observation to your agent here)
                if is_training and random.random() < epsilon: # If we are training the DQN and the random number generated by "random.random()" is less than epsilon, we will do a random action (means the agent will take a random action). When we just start the DQN training and epsilon=1, most likely we will enter this section (because the number generated by random() is between 0 to 1)
                    action = env.action_space.sample() # use sample() on action space(action_space) of the environment(env) to get a random action for the agent (flappy bird) at each iteration. This means the sample() might return different actions(values) at each iteration. 
                    # More explanation: 
                    # 1) The action_space of an environment refers to all possible actions that can be taken by the agent. In this environment, the action_space consists of only 2 values(0:the agent[flappy bird] do nothing; 1:the agent[flappy bird] flaps its wing to fly up). 
                    # 2) Hence, the sample() here will only return either 0 or 1. While the returned value is stored in the action variable.
                    
                    # Convert the variable state into tensor object
                    action = torch.tensor(action, dtype=torch.int64, device=device) # Since we are using PyTorch to develop DQN, we need to make sure the things that are going into the DQN are tensor object. In this tutorial, the things that are going into the DQN are the components of experience (state, action, new_state, reward, terminated). Since the variable action is the thing that goes into the DQN, we need to convert it into a tensor object, using the tensor function. After we passing the variable action into the tensor function, the variable action is converted into tensor object and now its values become 64-bit signed interger number (because dtype=torch.int64). Then, we will send the converted variable state to the device we selected (either CPU or GPU) for processing. 

                else: # otherwise, we will select the action that the DQN (policy network) prescribes
                    with torch.no_grad(): # here we are estimating for the best action. PyTorch does gradient calculation automatically during the DQN training. Since at here, we're not doing training, we're just evaluating a state (choosing the best action provided by the DQN), so we can turn off the gradient calculation of PyTorch using this line, just to save on processing power.
                        # we provide "state" as the input to the policy_dqn (the DQN), then policy_dqn will provide the Q values of all actions (In this environment, since there are only 2 actions in the action space, the DQN will only provide 2 Q values as the output). 
                        # Since we want the agent to take the action with the highest Q value & the index of Q values returned by the DQN represents an action [here, index 0 represents the agent do nothing; index 1 represents the agent flaps its wing], ".argmax()" is used to return the index of the highest Q value. 
                        # Hence, the index of the highest Q value is selected as the action prescribes by the DQN for the agent to take to move into next state from the current state.
                        action = policy_dqn(state.unsqueeze(dim=0)).squeeze().argmax() 
                        # "unsqueeze(dim=0)" is appied to variable state to add 1 more dimension at its very front (so now 1-dimensional tensor([1,2,3,...]) becomes 2-dimensional tensor([[1,2,3,...]])). The newly added dimension becomes the 1st dimension of the variable state that stores the information of batch.
                        # Since the input of the DQN, variable state now has become 2-dimensional, the output of the DQN also becomes 2-dimensional. The output of the DQN are the Q values of all actions in the action space. Hence, we apply "squeeze()" on the DQN outputs to squeenze the outputs into 1-dimensional, so now we can get the index of each Q value provided by the DQN.
                        


In [ ]:
# Obviously, at the beginning when the DQN is untrained, DQN is going to spit out garbage. But as we train the DQN, the policy gets better (means the DQN provide better outputs), and we'll get better actions from the policy (DQN).
            # So, we want to slowly decrease the epsilon after 1 episode, by performing epsilon multiplying with epsilon_decay (actually there are different methods to decrease the epsilon). We take the maximum value between the decreased epsilon and the minimum epsilon to make sure the epsilon does not go under the minimum.
            epsilon = max(epsilon * self.epsilon_decay, self.epsilon_min)
            # keep track of the epsilon history at each episode
            epsilon_history.append(epsilon)